In [1]:
from ExpDerive.nlp.nlp_api import NlpAPI
from ExpDerive.derive.resolve import ExpAPI

from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

import json

import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
columns_path = '/Users/danielfisaha/exp_derive/ExpDerive/data/epl_columns.json'
# columns = pd.read_json(columns_path)
with open(columns_path) as f:
    columns = json.load(f)
    df = pd.DataFrame(columns=[col['lable'] for col in columns])
    df.set_index('player')

df.head()


,ranker,player,nationality,position,team,age,birth_year,games,games_starts,minutes,...,assists_per90,goals_assists_per90,goals_pens_per90,goals_assists_pens_per90,xg_per90,xg_assist_per90,xg_xg_assist_per90,npxg_per90,npxg_xg_assist_per90,matches


In [3]:
path = '/Users/danielfisaha/exp_derive/ExpDerive/data/epl_stats.csv'
with open(path, 'r') as f:
    for line in f:
        stats = line.split(',')
        if stats[1] == 'Player':
            continue
        stats = stats[:-1]
        # print(stats)
        df = df.append(pd.Series(stats, index = df.columns), ignore_index=True)

/var/folders/rv/87hkf18j3g9dyl6v51hb5fdr0000gn/T/ipykernel_61205/1358045091.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(stats, index = df.columns), ignore_index=True)
/var/folders/rv/87hkf18j3g9dyl6v51hb5fdr0000gn/T/ipykernel_61205/1358045091.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(stats, index = df.columns), ignore_index=True)
/var/folders/rv/87hkf18j3g9dyl6v51hb5fdr0000gn/T/ipykernel_61205/1358045091.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(stats, index = df.columns), ignore_index=True)
/var/folders/rv/87hkf18j3g9dyl6v51hb5fdr0000gn/T/ipykernel_61205/1358045091.py:9: FutureWarning: The frame.append method is deprecate

In [3]:
def filter_columns(col):
    try:
        c = col['description']
        return True
    except KeyError:
        return False
    
columns = [col for col in columns if filter_columns(col)]
# columns

In [4]:
column_descriptions = [col['description'] for col in columns]
column_labels = [col['lable'] for col in columns]
# column_arr

In [5]:
# resolvers 
def var_resolver(plc):
    return {
        'name': plc,
        'leaf': True,
    }
def eval_resolver(player, stat):
    return df.loc[df['player'] == player][stat].values[0]


In [6]:
nlp = NlpAPI(api_key=os.getenv("OPENAI_API_KEY"), stat_type="football stats")
nlp.load_column_classifier('/Users/danielfisaha/exp_derive/ExpDerive/models/test_classifier.pkl')
# nlp.train_column_classifier(column_descriptions, column_labels)


In [8]:
# from copy import deepcopy
# import pickle
# test_classifier = deepcopy(nlp.column_classifier)


In [9]:
# pickle.dump(test_classifier, open('/Users/danielfisaha/exp_derive/ExpDerive/models/test_classifier.pkl', 'wb'))

In [10]:
# model = pickle.load(open('/Users/danielfisaha/exp_derive/ExpDerive/models/test_classifier.pkl', 'rb'))
# model.classify('goals for')

In [11]:
# nlp.column_classifier.classify('total goals scored')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


'goals'

In [7]:
request = 'goals scored plus expected goals divided by 2'
phrase = nlp.parse_phrase(request)
phrase.latex

Extract all of the distinct football stats from the following expression: goals scored plus expected goals divided by 2


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


g*(o*(a*(l*(s*(p*(l*(u*(s*((g*x)/2))))))))) before goalsplus \frac{xg}{2}
goalsplus*(xg/2) after \goalsplus \frac{\xg}{2}


'\\goalsplus \\frac{\\xg}{2}'

In [13]:
from sympy.parsing.latex import parse_latex

In [14]:
phrase.columns

[('Goals scored', 'goals'), ('Expected goals', 'xg')]

In [15]:
parse_latex(phrase.latex)

goals + xg/2

In [3]:
def f():
    for i in range(10):
        yield i

l = list(map(lambda x: print(x), [1,2,3]))
print(l)

1
2
3
[None, None, None]
